from: https://github.com/SadmanSakib93/Stratified-k-fold-cross-validation-Image-classification-keras/blob/master/stratified_K_fold_CV.ipynb

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2023-04-25 05:27:00.934521: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 05:27:04.479160: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
cwd = os.getcwd()
print("current working directory: ",cwd)

current working directory:  /workspaces/proposal-ta


In [3]:
"""from google.colab import drive
drive.mount('/content/drive')
!ls
import sys
# This is the path to where in google drive the code is stored!
root_path = '/content/drive/My Drive/'
sys.path.append(root_path)
"""

root_path = '/workspaces/proposal-ta/images/'

In [4]:
### UBAH NAMA FILE MODEL SESUAI PENGUJIAN DI SINI
dataset_folder_name = os.path.join(root_path, '')
source_files = []
class_labels = ['a', 'ba', 'ca', 'da', 'ga', 
                'ja', 'ka','la','ma', 'na', 
                'nga', 'nya','pa', 'ra', 'sa', 
                'ta', 'wa', 'ya']
X = []
Y = []

img_rows, img_cols = 120, 120 # input image dimensions
train_path = os.path.join(dataset_folder_name, 'train')
validation_path = os.path.join(dataset_folder_name, 'validation')
test_path = os.path.join(dataset_folder_name, 'test')

In [5]:
# augmentasi data
# bisa jadi sebelum atau setelah split

In [6]:
def transfer_between_folders(source, dest, split_rate):
    """ Based on the split ratio this function moves some portion of the source folder to destination folder!

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest location

    """
    global source_files
    source_files = os.listdir(source)
    if(len(source_files) != 0):
        transfer_file_numbers = int(len(source_files)*split_rate)
        transfer_index = random.sample(
            range(0, len(source_files)), transfer_file_numbers)
        for each_index in transfer_index:
            shutil.move(os.path.join(source, str(source_files[each_index])), os.path.join(
                dest, str(source_files[each_index])))

    else:
        print("No file moved. Source empty!")


def transfer_all_class_between_folders(source, dest, split_rate):
    """ Transfer the files from source to dest for all the classes. This function calls the 'transfer_between_folders' to actually perform the transfer.

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest location

    """
    for label in class_labels:
        transfer_between_folders(os.path.join(dataset_folder_name, source, label),
                                 os.path.join(
                                     dataset_folder_name, dest, label),
                                 split_rate)


def my_metrics(y_true, y_pred):
    """ Calculate accuracy, precision, and f1 score of the model's prediction with respect to true labels.

        Args:
            y_true: list/array
                All true class labels
            y_pred: list/array
                All predicted class labels

        Returns:
            accuracy: float
                Accuracy measure of the model
            precision: float
                Precision measure of the model
            f1_Score: float
                F1-score measure of the model

    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    f1_Score = f1_score(y_true, y_pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1_Score))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1_Score

In [7]:
transfer_all_class_between_folders('test','train', 1.0)
#transfer_all_class_between_folders('validation','train', 1.0)

In [8]:
transfer_all_class_between_folders('train', 'test', 0.20)

In [9]:
def prepare_name_with_labels(folder_name, dataset_type='train'):
    """ Prepare the file names (X) and the class labels (Y) from folder location of images.

        Args:
            folder_name: str
                Source folder's path

    """
    source_files = os.listdir(os.path.join(dataset_folder_name, dataset_type, folder_name))
    y_label = 0
    for i in range(len(class_labels)):
        if(folder_name == class_labels[i]):
            y_label = i
    for val in source_files:
        X.append(val)
        Y.append(y_label)

In [10]:
# Organize file names and class labels in X and Y variables
for i in range(len(class_labels)):
    prepare_name_with_labels(class_labels[i])

In [11]:
X = np.asarray(X)
Y = np.asarray(Y)

In [12]:
# arsitektur
batch_size = 32
epoch = 100
num_of_channels = 3
number_of_class_labels = len(class_labels)

In [13]:
"""
    Note that, this model structure is a very basic one. 
    To achieve better performance, you should change the model structure and hyperparameters according to your needs and data. 
    So, optimize the structure of the model!
"""

#def get_model():
#    activation_function = 'relu'
#
#    model = Sequential()
#    model.add(Conv2D(64, (3,3), padding='same',
#                     activation=activation_function, input_shape=(img_rows, img_cols, num_of_channels)))
#    model.add(Conv2D(64, (3,3), activation=activation_function))
#    model.add(MaxPooling2D(pool_size=(2, 2)))
#    #model.add(Dropout(0.25))
#
#    model.add(Conv2D(32, (3,3), padding='same',
#                     activation=activation_function))
#    model.add(Conv2D(32, (3,3), activation=activation_function))
#    model.add(MaxPooling2D(pool_size=(2, 2)))
#    #model.add(Dropout(0.25))
#
#    model.add(Conv2D(16, (3,3), padding='same',
#                     activation=activation_function))
#    model.add(Conv2D(16, (3,3), activation=activation_function))
#    model.add(MaxPooling2D(pool_size=(2, 2)))
#    #model.add(Dropout(0.25))
#
#    #model.add(Conv2D(64, (3,3), padding='same',
#    #                 activation=activation_function))
#    #model.add(MaxPooling2D(pool_size=(2, 2)))
#    #model.add(Dropout(0.25))
#    
#    #model.add(Conv2D(64, (3,3), padding='same',
#    #                 activation=activation_function))
#    #model.add(MaxPooling2D(pool_size=(2, 2)))
#    #model.add(Dropout(0.25))
#
#    #model.add(Conv2D(128, (3,3), padding='same',
#    #                 activation=activation_function))
#    #model.add(MaxPooling2D(pool_size=(2, 2)))
#    #model.add(Dropout(0.25))
#
#    model.add(Flatten())
#    model.add(Dense(128, activation=activation_function))
#    #model.add(Dropout(0.1))
#    #model.add(Dense(32, activation=activation_function))
#    #model.add(Dropout(0.1))
#    #model.add(Dense(16, activation=activation_function))
#    #model.add(Dropout(0.1))
#    model.add(Dense(number_of_class_labels, activation='softmax'))
#
#    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
#
#    model.compile(optimizer=optimizer, loss='categorical_crossentropy',
#                  metrics=['accuracy'])
#
#    return model
#
#
#model = get_model()
#model.summary()
#

'\n    Note that, this model structure is a very basic one. \n    To achieve better performance, you should change the model structure and hyperparameters according to your needs and data. \n    So, optimize the structure of the model!\n'

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(120, 120, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    #tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(18, activation='softmax')
])

optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005)

model.compile(
    #optimizer='adam', 
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 59, 59, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 28, 28, 32)        0

In [15]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X, Y)
fold_num = 0
for train_index, val_index in skf.split(X, Y):
    # First cut all images from validation to train (if any exists)
    transfer_all_class_between_folders('validation', 'train'
                                       , 1.0)
    fold_num += 1
    print("Results for fold", fold_num)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for each_index in range(len(X_val)):
        class_label = ''
        for i in range(len(class_labels)):
            if(Y_val[each_index] == i):
                class_label = class_labels[i]
        # Then, copy the validation images to the validation folder
        shutil.move(os.path.join(dataset_folder_name, 'train', class_label, X_val[each_index]),
                    os.path.join(dataset_folder_name, 'validation', class_label, X_val[each_index]))

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.20,
        fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)
    
    # fit model
    history = model.fit(train_generator,
                        epochs=epoch)

    predictions = model.predict(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    
    # evaluate validation performance
    print("***Performance on Validation data***")
    val_acc, val_prec, val_fScore = my_metrics(true_classes, y_predictions)

Results for fold 1
Found 1222 images belonging to 18 classes.
Found 91 images belonging to 18 classes.
Epoch 1/100


2023-04-25 05:27:18.254437: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


 1/39 [..............................] - ETA: 1:34 - loss: 3.0753 - accuracy: 0.0000e+00

2023-04-25 05:27:20.783047: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 29942784 exceeds 10% of free system memory.
2023-04-25 05:27:20.822518: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 57032704 exceeds 10% of free system memory.


 2/39 [>.............................] - ETA: 13s - loss: 2.9355 - accuracy: 0.1053     

...

39/39 [==============================] - 13s 336ms/step - loss: 0.0222 - accuracy: 0.9943
Epoch 2/100
39/39 [==============================] - 12s 315ms/step - loss: 0.0158 - accuracy: 0.9951
Epoch 3/100
39/39 [==============================] - 12s 306ms/step - loss: 0.0173 - accuracy: 0.9951
Epoch 4/100
39/39 [==============================] - 12s 307ms/step - loss: 0.0255 - accuracy: 0.9935
Epoch 5/100
39/39 [==============================] - 12s 301ms/step - loss: 0.0228 - accuracy: 0.9926
Epoch 6/100
39/39 [==============================] - 12s 306ms/step - loss: 0.0100 - accuracy: 0.9984
Epoch 7/100
39/39 [==============================] - 12s 305ms/step - loss: 0.0126 - accuracy: 0.9959
Epoch 8/100
39/39 [==============================] - 12s 309ms/step - loss: 0.0098 - accuracy: 0.9959
Epoch 9/100
39/39 [==============================] - 12s 313ms/step - loss: 0.0239 - accuracy: 0.9902
Epoch 10/100
39/39 [==============================] - 13s 321ms/step - loss: 0.0292 - accuracy

2023-04-25 08:12:14.993703: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - 0s 63ms/step
***Performance on Validation data***
Accuracy  : 1.0
Precision : 1.0
f1Score : 1.0
[[5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5]]
Results for fold 9
Found 1223 images belonging to 18 classes.
Found 90 images belonging to 18 classes.
Epoch 1/100


2023-04-25 08:12:15.383007: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


39/39 [==============================] - 13s 318ms/step - loss: 0.0273 - accuracy: 0.9910
Epoch 2/100
39/39 [==============================] - 12s 309ms/step - loss: 0.0264 - accuracy: 0.9918
Epoch 3/100
39/39 [==============================] - 12s 312ms/step - loss: 0.0117 - accuracy: 0.9959
Epoch 4/100
39/39 [==============================] - 12s 314ms/step - loss: 0.0150 - accuracy: 0.9926
Epoch 5/100
39/39 [==============================] - 12s 312ms/step - loss: 0.0099 - accuracy: 0.9959
Epoch 6/100
39/39 [==============================] - 12s 317ms/step - loss: 0.0150 - accuracy: 0.9959
Epoch 7/100
39/39 [==============================] - 12s 309ms/step - loss: 0.0076 - accuracy: 0.9967
Epoch 8/100
39/39 [==============================] - 12s 310ms/step - loss: 0.0090 - accuracy: 0.9959
Epoch 9/100
39/39 [==============================] - 12s 312ms/step - loss: 0.0360 - accuracy: 0.9894
Epoch 10/100
39/39 [==============================] - 12s 307ms/step - loss: 0.0136 - accuracy

In [ ]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False 
)
predictions = model.predict(test_generator, verbose=1) 
y_predictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

test_acc, test_prec, test_fScore = my_metrics(true_classes, y_predictions)

In [ ]:
from sklearn.metrics import classification_report
clr = classification_report(true_classes, y_predictions,
                            labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 
                            target_names=['a', 'ba','ca','da','ga','ja','ka','la','ma','na','nga','nya','pa','ra','sa','ta','wa','ya'])
print(clr)

In [ ]:
MODEL_FILENAME = root_path+"/modelpengujian27_3x3_arsitektur2.h5"
model.save(MODEL_FILENAME)